<a href="https://colab.research.google.com/github/viphoangdep/license-plate-recognition/blob/main/Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 19.9 MB/s eta 0:00:00


In [ ]:
!unzip /content/drive/MyDrive/dataset.zip -d /content/dataset


Archive:  /content/drive/MyDrive/dataset.zip
   creating: /content/drive/MyDrive/dataset/dataset/
  inflating: /content/drive/MyDrive/dataset/dataset/0000_00532_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_00532_b.txt  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_02187_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_02187_b.txt  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_05696_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_05696_b.txt  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_06886_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_06886_b.txt  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_08244_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0000_08244_b.txt  
  inflating: /content/drive/MyDrive/dataset/dataset/0001_05318_b.jpg  
  inflating: /content/drive/MyDrive/dataset/dataset/0001_05318_b.txt  
  inflating: /content/drive/MyDrive/dataset/datase

In [ ]:
import os
import shutil
import random

# Đường dẫn đến dataset
dataset_dir = '/content/dataset/dataset'
train_dir = '/content/dataset/train'
valid_dir = '/content/dataset/valid'

# Tạo thư mục cho train và valid
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# Tạo thư mục images và labels trong mỗi thư mục
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'labels'), exist_ok=True)

# Tỉ lệ chia
train_ratio = 0.8

# Lấy danh sách các tệp ảnh và nhãn
image_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg') or f.endswith('.png')]
label_files = [f for f in os.listdir(dataset_dir) if f.endswith('.txt')]

# Chia tệp
for image_file in image_files:
    if image_file.replace('.jpg', '.txt') in label_files or image_file.replace('.png', '.txt') in label_files:
        if random.random() < train_ratio:
            # Di chuyển tệp vào thư mục train
            shutil.move(os.path.join(dataset_dir, image_file), os.path.join(train_dir, 'images', image_file))
            shutil.move(os.path.join(dataset_dir, image_file.replace('.jpg', '.txt').replace('.png', '.txt')),
                        os.path.join(train_dir, 'labels', image_file.replace('.jpg', '.txt').replace('.png', '.txt')))
        else:
            # Di chuyển tệp vào thư mục valid
            shutil.move(os.path.join(dataset_dir, image_file), os.path.join(valid_dir, 'images', image_file))
            shutil.move(os.path.join(dataset_dir, image_file.replace('.jpg', '.txt').replace('.png', '.txt')),
                        os.path.join(valid_dir, 'labels', image_file.replace('.jpg', '.txt').replace('.png', '.txt')))

print("Đã chia dataset thành công thành train và valid.")


Đã chia dataset thành công thành train và valid.


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.6/873.6 kB 58.2 MB/s eta 0:00:00


In [ ]:
data_yaml_content = """
train: /content/dataset/train/images
val: /content/dataset/valid/images

nc: 1  # Số lớp (chỉ cần 1 cho biển số xe)
names: ['license_plate']  # Tên lớp
"""

with open('/content/dataset/data.yaml', 'w') as f:
    f.write(data_yaml_content)


In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv8 đã được huấn luyện trước
model = YOLO('yolov8n.pt')  # Có thể thay thế bằng yolov8s.pt, yolov8m.pt, v.v.

# Huấn luyện mô hình
model.train(data='/content/dataset/data.yaml', epochs=30, imgsz=480)  # imgsz là kích thước ảnh


100%|██████████| 6.25M/6.25M [00:00<00:00, 78.2MB/s]


Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataset/train/labels... 2001 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2001/2001 [00:01<00:00, 1945.97it/s]

...

                   all        489        489          1          1      0.995      0.839



30 epochs completed in 0.269 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:04<00:00,  3.81it/s]


                   all        489        489          1          1      0.995      0.839
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7708f71570>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
model.save('/content/drive/MyDrive/lisence_yolo_model.pt')  # Lưu mô hình với tên tuỳ chọn


In [ ]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/lisence_yolo_model.pt')

In [ ]:
image_path = '/content/dataset/valid/images/0002_02554_b.jpg'  # Thay đổi đường dẫn đến ảnh
results = model.predict(image_path)



image 1/1 /content/dataset/valid/images/0002_02554_b.jpg: 416x640 1 license_plate, 114.6ms
Speed: 12.9ms preprocess, 114.6ms inference, 1238.5ms postprocess per image at shape (1, 3, 416, 640)


In [ ]:
boxes = results[0].boxes

In [ ]:
import cv2

# Load the original image using OpenCV
img = cv2.imread('/content/dataset/valid/images/0002_02554_b.jpg' )

# Extract bounding box coordinates
for box in boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Lấy tọa độ x1, y1, x2, y2
    plate_img = img[y1:y2, x1:x2]  # Cắt phần biển số ra từ ảnh gốc


In [ ]:
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])  # 'en' cho tiếng Anh, nếu biển số là ký tự khác bạn có thể thay đổi

# Recognize text from the cropped image

result = reader.readtext(plate_img)
texts = [text for _, text, _ in result]
print(texts)

['59FI', '609.55']


In [ ]:
# Add the recognized plate text on the original image
cv2.putText(img, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Display the image with text
cv2.imshow('Image with Plate Number', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the image with the text
cv2.imwrite('output_with_plate_text.jpg', img)
